### 3D (Depthwise 2D) model training
I'm going to try working with the 3D images and run them through a basic Conv3D model, see what happens. I'm going to be using the resized data. I found this convolution function called depthwise2D which essentially takes the depth array as a lot of channels in the data.
Model name: conv-depthwise

#### Writeup of the different model trials:
conv-depthwise-0: Basic depthwise model, depthwiseconv2d: kernel_size=32, strides=(2,2), padding='same', global pooling, one hidden layer of size 40, batch_size=10, epochs=10 - training accuracy = 36.84%, best validation acc = 40.74%

conv-depthwise-1: Basic depthwise model, depthwiseconv2d: kernel_size=32, strides=(2,2), padding='same', global pooling, one hidden layer of size 40, batch_size=5, epochs=10 - best training accuracy = 36.84%, best val acc = 41.36%

conv-depthwise-2: Changed strides to (1,1). Changed padding to 'valid'. Changed learning rate to 0.005, from 0.05 in earlier iterations. Trying on reduced epochs (5) as in the past two iterations the accuracy plateaued by the first or second iteration. Best training accuracy = 36.84%, val acc = 40.74%

conv-depthwise-3: New data! (512, 512, 24) data shape, model - added bigger DNN (1024 -> 512 -> 5), added second layer of Depthwise Convolution with strides=(1,1)

conv-depthwise-4: Added two more convolutions of kernel size 64.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
X_train = np.load('data/manual_conv/X_train_full.npy')
y_train = np.load('data/manual_conv/y_train_full.npy')
X_test = np.load('data/manual_conv/X_test_full.npy')
y_test = np.load('data/manual_conv/y_test_full.npy')

In [3]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(132, 512, 512, 24) (34, 512, 512, 24) (132, 5) (34, 5)


In [8]:
checkpoint_path = "data/models/conv-depthwise-5-weights-improvement-{epoch:02d}-{acc:.2f}.hdf5"

In [9]:
import keras
from keras import layers as l
from keras.models import Model
from keras.callbacks import ModelCheckpoint

def create_model():
    input_img = l.Input(shape=(512, 512, 24))
    x = l.DepthwiseConv2D(kernel_size=32, strides=(1,1), padding='valid')(input_img)
    x = l.DepthwiseConv2D(kernel_size=32, strides=(2,2), padding='valid')(x)
    
    x = l.DepthwiseConv2D(kernel_size=64, strides=(1,1), padding='valid')(x)
    x = l.DepthwiseConv2D(kernel_size=64, strides=(2,2), padding='valid')(x)
    x = l.GlobalAveragePooling2D()(x)
    
    x = l.Dense(1024, activation='relu', name='hidden_layer_1')(x)
    x = l.Dense(512, activation='relu', name='hidden_layer_2')(x)
    output = l.Dense(5, activation='softmax', name='softmax_layer')(x)

    model = Model(inputs=input_img, outputs=output)
    return model


def train_depthwise(X=X_train, y=y_train, X_test=X_test, y_test=y_test, batch_size=10, epochs=10, verbose=1, path=checkpoint_path):
    model = create_model()
    rmsprop = keras.optimizers.RMSprop(lr=0.005)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    # checkpointing - saving in case something goes wrong
    filepath= path
    checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    model.fit(X, y, batch_size=batch_size, epochs=epochs, callbacks=callbacks_list, verbose=verbose)
    score = model.evaluate(X_test, y_test)
    return score

create_model().summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 512, 512, 24)      0         
_________________________________________________________________
depthwise_conv2d_1 (Depthwis (None, 481, 481, 24)      24600     
_________________________________________________________________
depthwise_conv2d_2 (Depthwis (None, 225, 225, 24)      24600     
_________________________________________________________________
depthwise_conv2d_3 (Depthwis (None, 162, 162, 24)      98328     
_________________________________________________________________
depthwise_conv2d_4 (Depthwis (None, 50, 50, 24)        98328     
_________________________________________________________________
global_average_pooling2d_1 ( (None, 24)                0         
_________________________________________________________________
hidd

In [ ]:
model = train(batch_size=10, epochs=5)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
132/132 [==============================] - 206s 2s/step - loss: 1.5148 - acc: 0.3636

Epoch 00001: acc improved from -inf to 0.36364, saving model to data/models/conv-depthwise-5-weights-improvement-01-0.36.hdf5
Epoch 2/5
132/132 [==============================] - 201s 2s/step - loss: 2.4243 - acc: 0.4394

Epoch 00002: acc improved from 0.36364 to 0.43939, saving model to data/models/conv-depthwise-5-weights-improvement-02-0.44.hdf5
Epoch 3/5
132/132 [==============================] - 201s 2s/step - loss: 2.4790 - acc: 0.3712

Epoch 00003: acc did not improve from 0.43939
Epoch 4/5
132/132 [==============================] - 204s 2s/step - loss: 1.8543 - acc: 0.4545

Epoch 00004: acc improved from 0.43939 to 0.45455, saving model to data/models/conv-depthwise-5-weights-improvement-04-0.45.hdf5
Epoch 5/5
120/132 [==========================>...] - ETA: 18s - loss: 1.4677 - acc: 0.3167

In [24]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1]) # want the accuracy score to be 0.9998

Test loss: 1.612279069774291
Test accuracy: 0.38235294117647056


In [17]:
model.save('data/models/conv-depthwise-4-best-model.hdf5')

In [25]:
# Here I'm checking the best validation accuracy model and it's testing accuracy
model2 = create_model()
model2.load_weights("data/models/conv-depthwise-4-weights-improvement-02-0.48.hdf5")
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [26]:
score = model2.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1]) # want the accuracy score to be 0.9998

Test loss: 1.2807262715171366
Test accuracy: 0.38235294117647056


### Multi-View CNN
Trying to see if I can implement a multi-view cnn in Keras, as described here: http://vis-www.cs.umass.edu/mvcnn/.
Right now our best accuracy is around 37.8%, with means data (we're able to get 37.8% both on the GDrive and here, though gdrive is a lot faster)

Model name:
conv-multiview

#### Writeup:
conv-multiview-0: Standard multiview cnn, separate 'towers' that did convolution and then global pooling, which are then concatenated and pushed into a deep network. Test acc- 36.84%, best val acc- 40.74% big whoop

conv-multiview-1: Made towers more complex with two Convolution layers split by Batch Normalization. 

conv-multiview-2: multiple conv layers per tower

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
X_train = np.load('data/manual_conv/X_train.npy')
y_train = np.load('data/manual_conv/y_train.npy')
X_test = np.load('data/manual_conv/X_test.npy')
y_test = np.load('data/manual_conv/y_test.npy')

In [4]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(162, 256, 256, 39) (19, 256, 256, 39) (162, 5) (19, 5)


In [5]:
checkpoint_path = "data/models/conv-multiview-2-weights-improvement-{epoch:02d}-{acc:.2f}.hdf5"

In [8]:
import keras
from keras import layers as l
from keras.models import Model
from keras.callbacks import ModelCheckpoint

def create_multiview_model(z_size=39):
    inputs_img = [l.Input(shape=(256, 256, 1), name=('input_' + str(i))) for i in range(0, z_size)]
    towers = []
    for img in inputs_img:
        x = l.Conv2D(32, (3, 3), padding='same', activation='relu')(img)
        x = l.BatchNormalization()(x)
                
        towers.append(x)
        
    x = l.concatenate(towers)
    x = l.GlobalAveragePooling2D()(x)
    
    x = l.Dense(40, activation='relu')(x)
    output = l.Dense(5, activation='softmax')(x)
    model = Model(inputs=inputs_img, outputs=output)
    return model


def train(X=X_train, y=y_train, batch_size=10, epochs=10, verbose=1, path=checkpoint_path):
    model = create_multiview_model()
    rmsprop = keras.optimizers.RMSprop(lr=0.005)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    # checkpointing - saving in case something goes wrong
    filepath= path
    checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    model.fit(X, y, batch_size=batch_size, epochs=epochs, callbacks=callbacks_list, verbose=verbose)
    return model

create_model().summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_0 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
input_4 (I

In [7]:
model = train(X=[X_train[:, :, :, i, np.newaxis] for i in range(0, 39)], y= y_train, batch_size=10, epochs=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1


ResourceExhaustedError: OOM when allocating tensor with shape[10,1248,256,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node concatenate_2/concat}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node metrics/acc/Mean}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [23]:
score = model.evaluate([X_test[:, :, :, i, np.newaxis] for i in range(0, 39)], y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1]) # want the accuracy score to be 0.9998

Test loss: 1.5489581823349
Test accuracy: 0.3684210479259491


### K-fold cross validation for depthwise Convolution models

In [2]:
import numpy as np
X_train = np.load('data/manual_conv/X_train_full.npy')
y_train = np.load('data/manual_conv/y_train_full.npy')
X_test = np.load('data/manual_conv/X_test_full.npy')
y_test = np.load('data/manual_conv/y_test_full.npy')

In [3]:
x = np.concatenate((X_train, X_test))
y = np.concatenate((y_train, y_test))
(x.shape, y.shape)

((166, 512, 512, 24), (166, 5))

In [4]:
y_kf = []
for i in range(0, y.shape[0]):
    y_val = "0"*y[i][0] + "1"*y[i][1] + "2a"*y[i][2] + "2b"*y[i][3] + "3"*y[i][4]
    y_kf.append(y_val)

In [5]:
y = y_kf

In [10]:
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
import pandas as pd

cv = StratifiedKFold(5)

acc = 0
j = 1
for train, test in cv.split(x, y):
    x_train = x[train]
    y_train = [y[i] for i in train]
    x_test = x[test]
    y_test = [y[i] for i in test]
    
    y_train = pd.get_dummies(y_train)
    y_test = pd.get_dummies(y_test)
    
    loss, accuracy = train_depthwise(x_train, y_train, x_test, y_test, 20, 10, path="data/models/conv-depthwise-kfold-0-weights-improvement-{epoch:02d}-{acc:.2f}.hdf5")
    print ('Fold #', j, 'Accuracy', accuracy)
    acc+=accuracy
    j+=1
print(acc/(j-1))

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


MemoryError: 

Accuracy levels are similar for this model, but - test losses (categorical crossentropy) are very low compared to other models we've tried so far.